In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
np.random.seed(666)
import time

physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Séparation des images en ensembles d'entraînement, validation et test

In [ ]:
import os, shutil 

original_dataset_dir= 'D:/Documents/Grad-CAM/oasis_brain_no_nn_slices/X/'

base_dir='D:/Documents/Grad-CAM/oasis_alzheimer_both/'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    
train_dir=os.path.join(base_dir,'train/')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    
validation_dir=os.path.join(base_dir,'validation/')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    
test_dir=os.path.join(base_dir,'test/')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

In [ ]:
train_alzheimer_dir=os.path.join(train_dir,'Alzheimer')
if not os.path.exists(train_alzheimer_dir):
    os.mkdir(train_alzheimer_dir)

train_healthy_dir = os.path.join(train_dir, 'Healthy')
if not os.path.exists(train_healthy_dir):
    os.mkdir(train_healthy_dir)
    
validation_alzheimer_dir=os.path.join(validation_dir,'Alzheimer')
if not os.path.exists(validation_alzheimer_dir):
    os.mkdir(validation_alzheimer_dir)

validation_healthy_dir = os.path.join(validation_dir, 'Healthy')
if not os.path.exists(validation_healthy_dir):
    os.mkdir(validation_healthy_dir)
    
test_alzheimer_dir=os.path.join(test_dir,'Alzheimer')
if not os.path.exists(test_alzheimer_dir):
    os.mkdir(test_alzheimer_dir)

test_healthy_dir = os.path.join(test_dir, 'Healthy')
if not os.path.exists(test_healthy_dir):
    os.mkdir(test_healthy_dir)

In [ ]:
file= open('D:/Documents/Grad-CAM/oasis_cross-sectional_labels.csv','r')

ligne= file.readlines()

file.close()

In [ ]:
noms = []
alzheimer = []


for i in range (1,len(ligne)):
    li=str(ligne[i]).rstrip('\n')
    liste=li.split(',')
    
    if liste[7] != "":
        noms.append(liste[0])
        if liste[7] == "0":
            alzheimer.append(0)
        else:
            alzheimer.append(1)

print(len(noms))
print(len(alzheimer))

In [ ]:
# Ajout d'images au dossier d'entrainement
for i in range(0, 175):
    if alzheimer[i] == 1:
        directory = "Alzheimer/"
        fname = noms[i] + '_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg'
        src= os.path.join(original_dataset_dir, *[directory, fname])
        dst=os.path.join(train_alzheimer_dir, fname)
        if os.path.isfile(src)==True:
            shutil.copyfile(src, dst)
    else:
        directory = "Healthy/"
        fname = noms[i] + '_mpr_nn_anon_111_t88_masked_gfchealthy.x77.jpg'
        src= os.path.join(original_dataset_dir, *[directory, fname])
        dst=os.path.join(train_healthy_dir, fname)
        if os.path.isfile(src)==True:
            shutil.copyfile(src, dst)

In [ ]:
# Ajout d'images au dossier de validation
for i in range(175, 200): 
    if alzheimer[i] == 1:
        directory = "Alzheimer/"
        fname = noms[i] + '_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg'
        src= os.path.join(original_dataset_dir, *[directory, fname])
        dst = os.path.join(validation_alzheimer_dir, fname)
        if os.path.isfile(src)==True:
            shutil.copyfile(src, dst)
    else:
        directory = "Healthy/"
        fname = noms[i] + '_mpr_nn_anon_111_t88_masked_gfchealthy.x77.jpg'
        src= os.path.join(original_dataset_dir, *[directory, fname])
        dst = os.path.join(validation_healthy_dir, fname)
        if os.path.isfile(src)==True:
            shutil.copyfile(src, dst)

In [ ]:
# Ajout d'images au dossier de test
for i in range(200, 235):
    if alzheimer[i] == 1:
        directory = "Alzheimer/"
        fname = noms[i] + '_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg'
        src= os.path.join(original_dataset_dir, *[directory, fname])
        dst = os.path.join(test_alzheimer_dir, fname)
        if os.path.isfile(src)==True:
            shutil.copyfile(src, dst)
    else:
        directory = "Healthy/"
        fname = noms[i] + '_mpr_nn_anon_111_t88_masked_gfchealthy.x77.jpg'
        src= os.path.join(original_dataset_dir, *[directory, fname])
        dst = os.path.join(test_healthy_dir, fname)
        if os.path.isfile(src)==True:
            shutil.copyfile(src, dst)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        #shear_range=0.2,
        zoom_range=0.2,
        brightness_range=[0.5,1.5],
        #horizontal_flip=True,
        #vertical_flip=True,
        #rotation_range=90
        )

In [ ]:
image_size = (150, 150)
batch_size = 2

training_set = train_datagen.flow_from_directory(
    "oasis_alzheimer_both/train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_set = validation_datagen.flow_from_directory(
    "oasis_alzheimer_both/validation",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    "oasis_alzheimer_both/test",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False
)

## Définition du modèle et entraînement

In [ ]:
densenet = tf.keras.applications.DenseNet201(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(150,150,3),
    pooling=None,
    classes=1000,
)

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

x = densenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

In [ ]:
densenet_model = Model(inputs=densenet.input, outputs=predictions)

In [ ]:
for layer in densenet.layers:
    layer.trainable = False

In [ ]:
densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
start_time = time.time()

history = densenet_model.fit(x = training_set, validation_data=validation_set, shuffle=True, epochs=20)

print(time.time() - start_time)

## Traçage des graphiques d'entraînement et ROC

In [ ]:
import matplotlib.pyplot as plt

acc= history.history['accuracy']
val_acc = history.history['val_accuracy']
loss= history.history['loss']
val_loss= history.history['val_loss']

epochs= range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='entrainement')
plt.plot(epochs, val_acc,'b', label='validation')
plt.title('Exactitude pendant l\'entrainement et la validation')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='entrainement')
plt.plot(epochs, val_loss, 'b', label='validation')
plt.title('Perte pendant l\'entrainement et la validation')
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc= densenet_model.evaluate(test_set)

In [ ]:
print(test_loss, test_acc)

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('./oasis_alzheimer_both/test/Alzheimer/OAS1_0430_MR1_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg', target_size = (150, 150))
test_image = image.img_to_array(test_image)/255.
test_image = np.expand_dims(test_image, axis=0)

prediction = densenet_model.predict_on_batch(test_image)
print(prediction)

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('./oasis_alzheimer_both/test/Alzheimer/OAS1_0432_MR1_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg', target_size = (150, 150))
test_image = image.img_to_array(test_image)/255.
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict_on_batch(test_image)
print(prediction)

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('./oasis_alzheimer_both/test/Healthy/OAS1_0426_MR1_mpr_nn_anon_111_t88_masked_gfchealthy.x77.jpg', target_size = (150, 150))
test_image = image.img_to_array(test_image)/255.
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict_on_batch(test_image)
print(prediction)

In [ ]:
y_true_labels = test_set.classes
true_labels = []

for label in y_true_labels:
    true_labels.append(np.array([np.float32(label)]))

preds = densenet_model.predict(test_set)

predictions = []

for x in preds.tolist():
    predictions.append(x[1])

from sklearn.metrics import roc_curve, auc

fpr, tpr, _ = roc_curve(true_labels, predictions)
auc = auc(fpr, tpr)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC (area = {:.3f})'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# Sauvegarde du modèle

In [ ]:
model.save('D:/Documents/Grad-CAM/models/OASIS/3_couches_adam')

### Pour juste loader le modèle

In [ ]:
import tensorflow as tf
from tensorflow import keras

loaded_model = keras.models.load_model('D:/Documents/Grad-CAM/models/OASIS/3_couches_adam_71_43_binary')

In [ ]:
test_loss, test_acc= loaded_model.evaluate(test_set)

# Grad-CAM

#### https://gist.github.com/RaphaelMeudec/e9a805fa82880876f8d89766f0690b54

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
IMAGE_PATH = './oasis_alzheimer_both/test/Alzheimer/OAS1_0388_MR1_mpr_nn_anon_111_t88_masked_gfcalzheimer.x77.jpg'

LAYER_NAME = 'conv3_block1_1_conv'
ALZHEIMER_CLASS_INDEX = 1
HEALTHY_CLASS_INDEX = 0

img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(150, 150))
img = tf.keras.preprocessing.image.img_to_array(img)/255.

#model = keras.models.load_model('D:/Documents/Grad-CAM/models/4_couches_adam_98_07_categorical')
model = densenet_model

grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(np.array([img]))
    loss = predictions[:, ALZHEIMER_CLASS_INDEX]

output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

gate_f = tf.cast(output > 0, 'float32')
gate_r = tf.cast(grads > 0, 'float32')
guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

weights = tf.reduce_mean(guided_grads, axis=(0, 1))

cam = np.ones(output.shape[0: 2], dtype = np.float32)

for i, w in enumerate(weights):
    cam += w * output[:, :, i]

cam = cv2.resize(cam.numpy(), (150, 150))
cam = np.maximum(cam, 0)
heatmap = (cam - cam.min()) / (cam.max() - cam.min())

cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

cv2.imwrite('cam.png', output_image)